# LSTM With doc2Vec

Transform each basket into a vector, then train a LSTM on the previous baskets to predict the next basket

In [2]:

import os
import collections

import random
import sqlite3
import numpy as np

# Load Data

In [1]:
def getOrders(lowerlimit, upperlimit):
    
    conn = sqlite3.connect("instacart.db")
    cur = conn.cursor()
    
    # Get final order
    cur.execute("SELECT B.user_id as user_id, A.order_id as order_id, "
                " B.order_number as order_number, A.product_id as product_id "
                "FROM products_train A INNER JOIN orders B "
                " ON A.order_id = B.order_id "
                "WHERE A.order_id % 100 >= " + str(lowerlimit) + \
                " AND A.order_id % 100 <= " + str(upperlimit) + ";")
    train_order = np.array(cur.fetchall())
    
    # Get all prior orders
    cur.execute( \
        "SELECT D.user_id as user_id, "
        "  D.order_id as order_id, "
        "  D.order_number as order_number, "
        "  C.product_id as product_id "
        "FROM products_prior C INNER JOIN ( "
        "  SELECT DISTINCT A.user_id as user_id,"
        "    A.order_id as order_id, A.order_number as order_number "
        "  FROM orders A INNER JOIN ( "
        "    SELECT DISTINCT user_id FROM orders "
        "    WHERE eval_set = 'train' "
        "      AND order_id % 100 >= " + str(lowerlimit) + 
        "      AND order_id % 100 <= " + str(upperlimit) +
        "    ) B ON A.user_id = B.user_id WHERE A.eval_set = 'prior' "
        ") D ON C.order_id = D.order_id;")
    prior_orders = np.array(cur.fetchall())
    
    conn.close()
    
    return train_order, prior_orders

In [44]:
y, x = getOrders(11, 19)

print(x.shape)
print(y.shape)

(1815695, 4)
(123907, 4)


# Train LSTM Model with Basket Vectors

In [4]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# create and fit the LSTM network
LSTM_model = Sequential()
LSTM_model.add(SimpleRNN(100, stateful=True, batch_input_shape=(100, 5, 49688), activation='tanh')) # input_shape=(batchsize, timesteps, data_dim)
LSTM_model.add(Dense(2000, activation='tanh'))
LSTM_model.add(Dropout(.25))
LSTM_model.add(Dense(49688, activation='sigmoid'))
LSTM_model.compile(loss='categorical_crossentropy', optimizer='adam')

Using TensorFlow backend.


## Create training data in batches of 500 orders

In [45]:
curPosition = 0

for n in range(5):
    print("Starting batch " + str(n) + " of ten.")
    print("--------------------------------")
    # Delete x_train / y_train from last iteration, may fail if this is first time through
    try:
        del y_train
    except:
        pass

    try:
        del x_train
    except:
        pass

    
    
    # i iterates over 500 orders 
    for i in range(100):
        
        # Print status
        if i % 25 == 0:
            print ("Working on order " + str(i) + " of this batch...")
        
        # New order id coming in:
        current_order_id = y[curPosition][1]
        current_user_id = y[curPosition][0]
        current_user_final_order_number = y[curPosition][2]
        
        # Initialize One Hot Encoding, one zero for each product
        current_user_y = np.zeros((49688))
        
        # This loop increments curPosition at the end   
        while current_order_id == y[curPosition][1]:
            
            # make the position in current_order_y = 1 for each product_id
            # Zero index is product_id == 1, so subtract 1 from the product id
            current_user_y[y[curPosition][3] - 1] = 1
        
            curPosition += 1
        

        # current_order_y is done, let's make the current_order_x: 
        # For reference: x and y have format: [user_id, order_id, order_number, product_id]

        
        # grab this user's prior orders
        # Only grab orders where the order number is in the last five orders
        prior_orders_products = x[np.where((x[:,0] == current_user_id) & (x[:,2] >= current_user_final_order_number - 5))]

        # initialize current_user_x
        current_user_x = np.zeros((5, 49688))

        # for each product in prior_orders_products, add to current_user_x
        for p in prior_orders_products:
            
            # current_user_x[a][b], where... 
            # a = timestep = The timestep is order_number - (current_user_final_order_number + 5)
            # b = product_id in OHE, minus 1 because of zero indexing
            current_user_x[p[2] - (current_user_final_order_number - 5)][p[3] - 1] = 1
    
        # Assert - current_user_x is done, add to x_train and y_train

        # Expand full order to batch dimension
        cur_user_x_train = np.expand_dims(current_user_x, axis=0)

        # Create x_train or add to it 
        try: 
            x_train = np.append(x_train, cur_user_x_train, axis=0)
        except:
            x_train = cur_user_x_train
            
        # do the same for y: 
        # Expand full order to batch dimension
        cur_user_y_train = np.expand_dims(current_user_y, axis=0)

        # Create x_train or add to it 
        try: 
            y_train = np.append(y_train, cur_user_y_train, axis=0)
        except:
            y_train = cur_user_y_train

    # Done creating batch, time to train
    LSTM_model.fit(x_train, y_train, epochs=15, batch_size=x_train.shape[0], verbose=2)
    

Starting batch 0 of ten.
--------------------------------
Working on order 0 of this batch...
Working on order 25 of this batch...
Working on order 50 of this batch...
Working on order 75 of this batch...
Epoch 1/15
2s - loss: 135.3825
Epoch 2/15
2s - loss: 134.9049
Epoch 3/15
2s - loss: 134.5703
Epoch 4/15
2s - loss: 134.0259
Epoch 5/15
2s - loss: 133.8054
Epoch 6/15
2s - loss: 133.2575
Epoch 7/15
2s - loss: 132.8877
Epoch 8/15
2s - loss: 132.5238
Epoch 9/15
2s - loss: 131.3139
Epoch 10/15
2s - loss: 130.7590
Epoch 11/15
2s - loss: 130.1371
Epoch 12/15
2s - loss: 129.2001
Epoch 13/15
2s - loss: 129.3345
Epoch 14/15
2s - loss: 128.2409
Epoch 15/15
2s - loss: 128.0906
Starting batch 1 of ten.
--------------------------------
Working on order 0 of this batch...
Working on order 25 of this batch...
Working on order 50 of this batch...
Working on order 75 of this batch...
Epoch 1/15
2s - loss: 121.2942
Epoch 2/15
2s - loss: 120.5892
Epoch 3/15
2s - loss: 120.0957
Epoch 4/15
2s - loss: 119.

In [ ]:
#LSTM_model.fit(x_train, y_train, epochs=15, batch_size=x_train.shape[0], verbose=2)


# Score the validation orders

## Grab validation orders:

In [100]:
# Get validation orders:
y_validation, x_validation = getOrders(70, 75)

## Change LSTM Model to accept one order at a time

In [46]:
# re-define model
LSTM_model_predict = Sequential()
LSTM_model_predict.add(SimpleRNN(100, stateful=True, batch_input_shape=(1, 5, 49688), activation='tanh')) # input_shape=(batchsize, timesteps, data_dim)
LSTM_model_predict.add(Dense(2000, activation='tanh'))
LSTM_model_predict.add(Dropout(.25))
LSTM_model_predict.add(Dense(49688, activation='relu'))

# copy weights
old_weights = LSTM_model.get_weights()
LSTM_model_predict.set_weights(old_weights)

# compile model
LSTM_model_predict.compile(loss='categorical_crossentropy', optimizer='adam')


## Make Function to predict for a given order

In [122]:
# New order id coming in:
current_order_id = y[curPosition][1]
current_user_id = y[curPosition][0]
current_user_final_order_number = y[curPosition][2]

# Initialize One Hot Encoding, one zero for each product
current_user_y = np.zeros((49688))


In [123]:
# New order id coming in:
current_order_id = y[curPosition][1]
current_user_id = y[curPosition][0]
current_user_final_order_number = y[curPosition][2]

# Initialize One Hot Encoding, one zero for each product
current_user_y = np.zeros((49688))

# This loop increments curPosition at the end   
while current_order_id == y[curPosition][1]:

    # make the position in current_order_y = 1 for each product_id
    # Zero index is product_id == 1, so subtract 1 from the product id
    current_user_y[y[curPosition][3] - 1] = 1

    curPosition += 1


# current_order_y is done, let's make the current_order_x: 
# For reference: x and y have format: [user_id, order_id, order_number, product_id]


# grab this user's prior orders
# Only grab orders where the order number is in the last five orders
prior_orders_products = x[np.where((x[:,0] == current_user_id) & (x[:,2] >= current_user_final_order_number - 5))]

# initialize current_user_x
current_user_x = np.zeros((5, 49688))

# for each product in prior_orders_products, add to current_user_x
for p in prior_orders_products:

    # current_user_x[a][b], where... 
    # a = timestep = The timestep is order_number - (current_user_final_order_number + 5)
    # b = product_id in OHE, minus 1 because of zero indexing
    current_user_x[p[2] - (current_user_final_order_number - 5)][p[3] - 1] = 1

# Assert - current_user_x is done, add to x_train and y_train

# Expand full order to batch dimension
cur_user_x_train = np.expand_dims(current_user_x, axis=0)

x_train = cur_user_x_train





In [144]:
print(current_user_id)
print(x_train.shape)
print(current_user_y.shape)

60568
(1, 5, 49688)
(49688,)


In [124]:
# Make Prediction
prediction = LSTM_model_predict.predict(x_train)

In [125]:
for i in range(prediction.shape[1]):
    print(prediction[0][i])

0.00311576
0.00334134
0.00315545
0.00324354
0.0032399
0.00315067
0.00321208
0.00320666
0.00309272
0.00331573
0.00328046
0.00317233
0.00323349
0.00317732
0.00343976
0.00309061
0.00318644
0.00316342
0.00318977
0.00333954
0.00330867
0.0033127
0.00335512
0.00320766
0.00414409
0.00334183
0.00310225
0.00328954
0.003234
0.00303187
0.00335372
0.00316144
0.00323422
0.00411309
0.00313714
0.00335542
0.00330798
0.00306701
0.00318689
0.00325515
0.00322784
0.0033038
0.00317807
0.00329027
0.00499882
0.00319503
0.00321048
0.00315554
0.00350435
0.0032622
0.00313723
0.00324246
0.00327503
0.00425632
0.00317058
0.00318721
0.00324239
0.00337886
0.00321781
0.00314103
0.00321835
0.00324948
0.00341578
0.0033645
0.0031286
0.00338155
0.00308846
0.00334153
0.00309874
0.00345602
0.00343733
0.0032363
0.00326323
0.00322568
0.00326511
0.0033461
0.00337479
0.00300445
0.00372899
0.0031486
0.00328236
0.00325057
0.00316371
0.00343486
0.003182
0.00330943
0.00310275
0.00326301
0.00317875
0.00312974
0.00317007
0.00324011
0

0.0032353
0.00335909
0.00346635
0.00322123
0.00310154
0.00341581
0.00323427
0.00419652
0.00376607
0.00313322
0.00323927
0.00337621
0.00335912
0.00326322
0.00413955
0.00333868
0.0033042
0.00332297
0.0031789
0.00307801
0.00307788
0.00318205
0.00336174
0.00335034
0.003178
0.00344584
0.00331596
0.00354832
0.00370524
0.00331168
0.00322951
0.00329048
0.00319053
0.00333519
0.00319596
0.00386956
0.00329601
0.00319676
0.00327857
0.00324851
0.00318857
0.00334216
0.00333963
0.00325961
0.00444953
0.00310838
0.00374835
0.00345239
0.00318799
0.00310778
0.00330196
0.00336331
0.00346971
0.00346362
0.00336963
0.00328162
0.00449998
0.00322784
0.00326847
0.00320186
0.00335252
0.00303832
0.00315693
0.00310992
0.0032607
0.00320423
0.00335246
0.00322828
0.00320709
0.00332674
0.00330739
0.00319451
0.00330268
0.00312166
0.00309761
0.00309825
0.00331888
0.00332265
0.00324578
0.00308206
0.00324289
0.00331153
0.00312198
0.0033427
0.00315361
0.00333065
0.00344617
0.00346504
0.00324098
0.00313243
0.00328496
0.0033

0.00309815
0.00330639
0.00325928
0.00311617
0.00311622
0.00310338
0.00318152
0.00433501
0.00322907
0.00339181
0.00316335
0.00324028
0.00330197
0.00335787
0.00325162
0.00322412
0.00389413
0.0031504
0.00326865
0.00337954
0.00314482
0.00313528
0.00331014
0.00306955
0.00417606
0.00529659
0.00315975
0.00317217
0.00329782
0.00319333
0.0033578
0.00332754
0.00332774
0.0032123
0.0032326
0.0032001
0.00319119
0.00320264
0.00355731
0.00328003
0.00343464
0.00333685
0.00375033
0.00324067
0.00321964
0.00332158
0.00330672
0.00310016
0.00340655
0.00332632
0.00327102
0.00321128
0.00318487
0.00317697
0.00327731
0.00323665
0.00335938
0.00322514
0.00325366
0.00319397
0.00310865
0.00313362
0.00328191
0.00328696
0.00324148
0.00336454
0.00327534
0.00302341
0.00317175
0.0032084
0.00327517
0.00319949
0.00322179
0.0032053
0.00321405
0.0031564
0.00314963
0.00322343
0.00321426
0.0032575
0.00330444
0.00342841
0.00322581
0.00311502
0.00331523
0.00322731
0.00317032
0.00535665
0.00325481
0.00316355
0.00340598
0.003301

0.00325408
0.00323963
0.00308342
0.00334395
0.00315256
0.00318376
0.00376118
0.00321661
0.00309399
0.00315976
0.00318299
0.00311779
0.00313525
0.00323948
0.00331658
0.00337138
0.00322333
0.00326224
0.00320568
0.00312691
0.00331688
0.00313898
0.00312309
0.0108364
0.00325701
0.0033025
0.00322648
0.00332595
0.00317281
0.00326916
0.00302134
0.00315715
0.0032469
0.00342624
0.0032942
0.0031547
0.0031734
0.00316917
0.00328604
0.00327644
0.00334504
0.00325871
0.00338501
0.00335697
0.0031995
0.00342631
0.0032491
0.00320555
0.00324231
0.00330491
0.00303182
0.00328318
0.00342723
0.0032513
0.0031222
0.00330886
0.0032824
0.00319059
0.00304025
0.00323868
0.0033582
0.00311347
0.003203
0.00323433
0.0032325
0.00310111
0.00315838
0.00323389
0.00302709
0.00331213
0.00325502
0.00317055
0.00306228
0.00316469
0.00318592
0.00311342
0.00321215
0.00338808
0.00331674
0.00330206
0.0032562
0.00328582
0.00315692
0.00415061
0.00327077
0.00316409
0.00316866
0.00347192
0.00327397
0.00320635
0.00332515
0.0034156
0.003

0.00323999
0.00336302
0.00324339
0.00327449
0.00312873
0.00330249
0.00327395
0.00330308
0.00327425
0.00335453
0.00335903
0.00317717
0.00322384
0.00345353
0.00322294
0.00315479
0.00345249
0.00331464
0.00330301
0.00322088
0.00311043
0.00316032
0.00633588
0.0034279
0.00347305
0.00312931
0.00347516
0.00319136
0.00624063
0.00311491
0.00340179
0.00326365
0.00326113
0.00326392
0.00321474
0.00320865
0.0033462
0.00327165
0.00314999
0.00334052
0.00329824
0.00316049
0.00320221
0.00313566
0.00324408
0.0031021
0.00303985
0.00329665
0.00330969
0.00332016
0.00476913
0.00336802
0.00321188
0.00319033
0.0033347
0.00318958
0.00507056
0.00319673
0.0032599
0.00333075
0.00326191
0.0030972
0.00421658
0.00400813
0.00326889
0.0033775
0.00342396
0.00335274
0.00312172
0.00332158
0.00326606
0.00325348
0.0031654
0.00309571
0.00329625
0.0033518
0.00328031
0.00311768
0.00304653
0.00321794
0.00317379
0.00318411
0.00312048
0.00321132
0.00329538
0.00331528
0.00312223
0.00311326
0.00307481
0.00359465
0.00338313
0.003174

...

0.00318235
0.0033647
0.00315427
0.00328012
0.00316785
0.00303382
0.00336147
0.0032039
0.00325114
0.0105148
0.0032693
0.00314949
0.00325572
0.0033291
0.00356874
0.00325237
0.0030427
0.00314248
0.00329245
0.00319922
0.00330259
0.00313946
0.0032847
0.00336258
0.00324333
0.00324164
0.00308936
0.00325571
0.00320166
0.00314685
0.00330586
0.00308848
0.00308265
0.00321529
0.00313699
0.00318326
0.00344202
0.00330196
0.00326537
0.00315076
0.00302705
0.00325621
0.00336476
0.00312001
0.00317983
0.014484
0.00339632
0.0035603
0.00328121
0.00315063
0.00404586
0.00319685
0.00348193
0.00352849
0.00314363
0.00332108
0.00321383
0.00323596
0.00339241
0.00340598
0.003239
0.00331079
0.00318402
0.00311129
0.00319532
0.00306705
0.00316279
0.00326625
0.00326063
0.00313658
0.00313116
0.00321635
0.00334423
0.00341407
0.00327326
0.00336201
0.00317458
0.00591587
0.00348621
0.00322509
0.00330312
0.00308462
0.0031091
0.00324616
0.00336081
0.00321772
0.00341279
0.00322396
0.00321959
0.00306647
0.00326114
0.00319818
0

0.00336918
0.00316464
0.00340825
0.00314755
0.00321642
0.00357626
0.00318179
0.00315581
0.00338466
0.00322395
0.00354137
0.00315876
0.0033416
0.0040365
0.00319784
0.00336295
0.00314048
0.00324124
0.00331727
0.00312864
0.0032618
0.00322965
0.00328018
0.0033034
0.00328908
0.00323938
0.00335086
0.00340873
0.00313658
0.00313379
0.00379972
0.00316009
0.00325898
0.0032772
0.0032563
0.00330854
0.00326328
0.00322431
0.00337138
0.00334932
0.00463684
0.0032576
0.00331824
0.00322945
0.00332836
0.00331148
0.00324973
0.00318548
0.00322743
0.00328684
0.00330187
0.00338507
0.003323
0.00337593
0.00326355
0.00327719
0.00335956
0.00330629
0.00323965
0.00387168
0.0032435
0.00312063
0.00339232
0.0032034
0.00353369
0.00344674
0.00318559
0.00320584
0.00330715
0.00327856
0.00329671
0.00322997
0.00345274
0.00326239
0.00407641
0.00320336
0.00335977
0.00322521
0.00323955
0.00332247
0.00322415
0.00304971
0.0033432
0.00328161
0.00322937
0.00309569
0.00386348
0.00313718
0.00327362
0.00328811
0.00326809
0.00335046


0.00336881
0.00334442
0.00325869
0.00333845
0.00313082
0.00318702
0.00326341
0.00314704
0.00324901
0.00326471
0.00338837
0.00330251
0.0030405
0.00338561
0.00328406
0.0034113
0.00307312
0.00417704
0.00332468
0.00338804
0.00306655
0.00324186
0.00323666
0.00321509
0.003141
0.00346605
0.00310294
0.00322336
0.00323617
0.00324706
0.00332402
0.00321772
0.00327055
0.00316328
0.00312178
0.00318143
0.00322967
0.00317127
0.00325425
0.00328288
0.00314996
0.00317617
0.00312471
0.00329375
0.00308057
0.00314569
0.00332164
0.00388199
0.00318945
0.00325629
0.00330846
0.00318363
0.00293595
0.00320848
0.00304215
0.0032155
0.00335405
0.00347551
0.00328973
0.00334305
0.00342102
0.00325602
0.00325345
0.00322185
0.00316489
0.00329454
0.00302009
0.00384255
0.00309232
0.0032343
0.00328729
0.00308424
0.00314394
0.00320256
0.00381283
0.00334824
0.00313203
0.00350863
0.00333887
0.00312223
0.00313507
0.00328497
0.0032913
0.00320888
0.0033088
0.00330215
0.00321177
0.00378061
0.00324324
0.00332184
0.00318417
0.00317

0.00331622
0.00328012
0.00331246
0.00338269
0.00350948
0.00322872
0.0031002
0.00326528
0.00321036
0.00318097
0.00341037
0.00326128
0.00317061
0.00318675
0.00478651
0.0037691
0.00344057
0.00339776
0.00312709
0.00332558
0.00332336
0.00319766
0.00302703
0.00332202
0.00337527
0.0033341
0.00324483
0.00318242
0.00314059
0.0031705
0.0032754
0.00330709
0.00311199
0.003403
0.00331112
0.00312172
0.00331351
0.0033217
0.00321486
0.0032607
0.00340133
0.00354696
0.00314295
0.00328957
0.00317932
0.00329745
0.00330184
0.00337669
0.00326395
0.00329881
0.0030579
0.00319044
0.00309532
0.00324797
0.00322714
0.00311895
0.0031589
0.00310452
0.00297794
0.00311919
0.00309662
0.00326201
0.00330097
0.00319181
0.00314747
0.00328872
0.00337759
0.00322171
0.0054485
0.00324368
0.00396015
0.00330497
0.00411502
0.00309238
0.0033389
0.00334744
0.00312591
0.00354903
0.00312668
0.00383189
0.00316522
0.00312067
0.00325449
0.0034306
0.00330486
0.00324184
0.00318169
0.00335214
0.00310207
0.00318296
0.00317192
0.00311722
0.

0.00325099
0.00319473
0.00352587
0.00318538
0.00327517
0.00326419
0.00328762
0.00326838
0.00314935
0.00345858
0.00350448
0.00330684
0.00325978
0.00320573
0.00323417
0.0032484
0.00325999
0.00333049
0.00327224
0.00321594
0.00355922
0.0030111
0.00328656
0.00318103
0.00315241
0.00318003
0.00320496
0.00316701
0.00323813
0.00322972
0.00324916
0.00329308
0.00353022
0.00327416
0.0266593
0.00326545
0.00325944
0.00396
0.00426784
0.00314723
0.00323062
0.00316286
0.00319594
0.00321759
0.00336691
0.00307708
0.00317025
0.00336744
0.00311185
0.00319745
0.00335368
0.00317114
0.00459182
0.00328028
0.00325182
0.00307924
0.00321876
0.00330744
0.00333017
0.00322232
0.00322948
0.00357552
0.00365994
0.00312971
0.00342341
0.00312833
0.00321318
0.00338126
0.00326055
0.00326649
0.00312613
0.00322621
0.00317
0.00316991
0.00314746
0.0041979
0.0033278
0.00319983
0.00349641
0.00340349
0.00331715
0.00344062
0.00330581
0.00324142
0.00344955
0.0032367
0.00323305
0.0031245
0.00364994
0.0031928
0.00306012
0.0032166
0.0

In [138]:
for i in range(current_user_y.shape[0]):
    if current_user_y[i] > 0:
        print("product_id is " + str(i))
        print("The prediction was " + str(prediction[0][i]))


product_id is 21076
The prediction was 0.00326622
product_id is 48363
The prediction was 0.00986695


In [134]:
prediction.mean()
prediction.std()

0.0013744554

In [140]:
m = prediction.max()
s = prediction.mean()

for i in range(prediction.shape[1]):
    if prediction[0][i] > m - s - s - s:
        print("product_id = " + str(i) + " - the prediction was " + str(prediction[0][i]))

product_id = 24851 - the prediction was 0.166191


In [146]:
print(current_order_id)

3278270


In [7]:
def getPrevPurchProds(lowerbound, upperbound):
    conn = sqlite3.connect("instacart.db")
    cur = conn.cursor()
    
    # Get list of products
    cur.execute("SELECT DISTINCT D.user_id as user_id, C.product_id as product_id "
                "FROM products_prior C INNER JOIN ( "
                "  SELECT A.user_id as user_id, B.order_id as order_id "
                "  FROM orders A INNER JOIN orders B ON A.user_id = B.user_id "
                "  WHERE A.order_id % 100 >= " + str(lowerbound) + \
                "    AND A.order_id % 100 <= " + str(upperbound) + " AND A.eval_set = 'train' ) D "
                " ON C.order_id = D.order_id;")
    previous_buys = np.array(cur.fetchall())
    
    conn.close()
    
    user_products = dict()
    
    for row in previous_buys:
        try:
            user_products[row[0]].add(row[1])
        except:
            user_products[row[0]] = set()
            user_products[row[0]].add(row[1])
            
    return user_products

In [172]:
# Load F1 scoring
%run F1_Score.ipynb 

# Create a method to calculate F1 for models so we can tune hyperparameters
# Assumes model takes input shape (1, 5, 49688)
# The model needs to be prepared to accept one order at a time! 
def calcF1(the_model, lowerbound, upperbound, threshold=0.2):
    
    # Get data
    print("getting orders...")
    %time y, x = getOrders(lowerbound, upperbound)
    
    # Get previously purchased products
    print("\ngetting previously purchased products")
    %time rebuy_dict = getPrevPurchProds(lowerbound, upperbound)
    
    ###
    # Initialize dictionaries to hold stuff! 
    ###

    # actual_dict is all products in actual purchase, keyed by order_id
    actual_dict = dict()

    # scored_dict is just the products they've repurchased (excludes new items not purchased previously)
    # keyed by order_id
    scored_dict = dict()
    
    # predicted_rebuys, keyed by order_id, only has items the model predicts they'll repurchase
    predicted_rebuys = dict()
    
    # predicted_allbuys holds all items the model thinks they bought
    predicted_allbuys = dict()

    # y_to_user_and_number makes a record for each order in the output set
    y_to_user_and_number = dict()
    
    # Create dictionary for each order in y:
    print("\nCreating dictionary for each order in y...")
    for row in y:
        try:
            actual_dict[row[1]].add(row[3])
        except:
            actual_dict[row[1]] = set([row[3]])
            y_to_user_and_number[row[1]] = [row[0], row[2]]
        
        # Create dictionary for repurchases in y:
        try:
            if row[3] in rebuy_dict[row[1]]:
                scored_dict[row[1]].add(row[3])
        except:
            scored_dict[row[1]] = set([row[3]])

    # One last thing - gotta make sure we have "None" if there are no rebuys.
    # Using actual_dict, go check for empty scored_dict etries
    for order in list(actual_dict.keys()):
        try:
            rebuySize = len(scored_dict[order])
        except:
            scored_dict[order] = set(["None"])

    # ASSERT - scored_dict is now keyed by order_id, gives products repurchased
    #        actual_dict is keyed by order_id, gives all products purchaesd (not just rebuys)
    #        y_to_user_and_number is keyed by order_id, gives user_id and order_number

        
    print("\nStarting to work through predicting orders, there are " + str(len(actual_dict)) + " orders...")
      
    progressCounter = 0
        
    # for each order in actual_dict, go recreate a OHE for the last five orders and predict
    for order in list(actual_dict.keys()):
        
        # grab this user's prior orders
        # Only grab orders where the order number is in the last five orders
        # See how clever I am using y_to_user_and_number? Boom. 
        prior_orders_products = x[np.where((x[:,0] == y_to_user_and_number[order][0]) & \
                                    (x[:,2] >= y_to_user_and_number[order][1] - 5))]

        # initialize current_user_x
        current_user_x = np.zeros((5, 49688))

        # for each product in prior_orders_products, add to current_user_x
        for p in prior_orders_products:
            
            # current_user_x[a][b], where... 
            # a = timestep = The timestep is order_number - (current_user_final_order_number + 5)
            # b = product_id in OHE, minus 1 because of zero indexing
            current_user_x[p[2] - (y_to_user_and_number[order][1] - 5)][p[3] - 1] = 1
    
        # Assert - current_user_x is done, add to x_train and y_train
        prediction = the_model.predict(x_train)
        
        # Create a set for this order in our predicted dictionaries
        predicted_rebuys[order] = set()
        predicted_allbuys[order] = set()
        
        # prediction is a numpy array of all the products, time to predict which they rebuy
        # First, the probability has to be beyond the threshold
        # then, check if the item was a repurchase
        for i in range(prediction.shape[1]):

            if prediction[0][i] > threshold:
            
                # don't forget that prediction is zero indexed, and product ids start at 1. Thus, add one to i 
                predicted_allbuys[order].add(i + 1)
              
                if i+1 in rebuy_dict[y_to_user_and_number[order][0]]:
                    predicted_rebuys[order].add(i+1)

        
        # Okay! Now, just handle if nothing gets bought:
        if len(predicted_rebuys[order]) == 0:
            predicted_rebuys[order].add("None")
        
        progressCounter += 1
        if progressCounter % 100 == 0:
            print("\nJust finished order " + str(progressCounter))
    
    # Double okay! I have everything I need, let's call F1 twice:
    #print("Calculating Rebuy F1 Score...")
    #f1(predicted_rebuys, scored_dict)
    #print("-----------------------------")
    #print("Calculating all purchase F1 score...")
    #f1(predicted_allbuys, actual_dict)
    return predicted_rebuys, predicted_allbuys, scored_dict, actual_dict

In [111]:
# God help me, this is not gonna work
pr, pa, ar, aa = calcF1(LSTM_model_predict, 70, 70, 0.2)

getting orders...
CPU times: user 9.69 s, sys: 880 ms, total: 10.6 s
Wall time: 11.2 s

getting previously purchased products
CPU times: user 2.03 s, sys: 1.04 s, total: 3.07 s
Wall time: 10.6 s

Creating dictionary for each order in y...

Starting to work through predicting orders, there are 1270 orders...

Just finished order 100

Just finished order 200

Just finished order 300

Just finished order 400

Just finished order 500

Just finished order 600

Just finished order 700

Just finished order 800

Just finished order 900

Just finished order 1000

Just finished order 1100

Just finished order 1200


In [8]:
# Get data
print("getting orders...")
%time y, x = getOrders(70, 70)

# Get previously purchased products
print("\ngetting previously purchased products")
%time rebuy_dict = getPrevPurchProds(70, 70)

###
# Initialize dictionaries to hold stuff! 
###

# actual_dict is all products in actual purchase, keyed by order_id
actual_dict = dict()

# scored_dict is just the products they've repurchased (excludes new items not purchased previously)
# keyed by order_id
scored_dict = dict()

# predicted_rebuys, keyed by order_id, only has items the model predicts they'll repurchase
predicted_rebuys = dict()

# predicted_allbuys holds all items the model thinks they bought
predicted_allbuys = dict()

# y_to_user_and_number makes a record for each order in the output set
y_to_user_and_number = dict()

# Create dictionary for each order in y:
print("\nCreating dictionary for each order in y...")
for row in y:
    try:
        actual_dict[row[1]].add(row[3])
    except:
        actual_dict[row[1]] = set([row[3]])
        y_to_user_and_number[row[1]] = [row[0], row[2]]

    # Create dictionary for repurchases in y:
    try:
        if row[3] in rebuy_dict[row[1]]:
            scored_dict[row[1]].add(row[3])
    except:
        scored_dict[row[1]] = set([row[3]])

# One last thing - gotta make sure we have "None" if there are no rebuys.
# Using actual_dict, go check for empty scored_dict etries
for order in list(actual_dict.keys()):
    try:
        rebuySize = len(scored_dict[order])
    except:
        scored_dict[order] = set(["None"])

# ASSERT - scored_dict is now keyed by order_id, gives products repurchased
#        actual_dict is keyed by order_id, gives all products purchaesd (not just rebuys)
#        y_to_user_and_number is keyed by order_id, gives user_id and order_number



getting orders...
CPU times: user 9.53 s, sys: 800 ms, total: 10.3 s
Wall time: 11 s

getting previously purchased products
CPU times: user 2 s, sys: 1.01 s, total: 3.01 s
Wall time: 10.5 s

Creating dictionary for each order in y...


In [47]:
thecounter = 0

predictions = dict()

avgPrediction = np.zeros((49688))

for order in list(actual_dict.keys()):
    if thecounter > 5:
        break

    # grab this user's prior orders
    # Only grab orders where the order number is in the last five orders
    # See how clever I am using y_to_user_and_number? Boom. 
    prior_orders_products = x[np.where((x[:,0] == y_to_user_and_number[order][0]) & \
                                (x[:,2] >= y_to_user_and_number[order][1] - 5))]

    # initialize current_user_x
    current_user_x = np.zeros((5, 49688))

    # for each product in prior_orders_products, add to current_user_x
    for p in prior_orders_products:

        # current_user_x[a][b], where... 
        # a = timestep = The timestep is order_number - (current_user_final_order_number + 5)
        # b = product_id in OHE, minus 1 because of zero indexing
        current_user_x[p[2] - (y_to_user_and_number[order][1] - 5)][p[3] - 1] = 1

        
    # Transform to correct dimension
    x_predict = np.expand_dims(current_user_x, axis=0)
    
    # Assert - current_user_x is done, add to x_train and y_train
    predictions[thecounter] = LSTM_model_predict.predict(x_predict).flatten()
    avgPrediction = avgPrediction + predictions[thecounter]
    print("Order: " + str(order) + ", avg: " + str(predictions[thecounter].mean()) + ", max: " + str(predictions[thecounter].max()))

    # Create a set for this order in our predicted dictionaries
    #predicted_rebuys[order] = set()
    #predicted_allbuys[order] = set()
    thecounter += 1


/opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Order: 170, avg: 0.00100285, max: 0.644872
Order: 10270, avg: 0.00123064, max: 0.839513
Order: 12770, avg: 0.00183175, max: 1.24715
Order: 14170, avg: 0.00234471, max: 1.62985
Order: 14970, avg: 0.00212756, max: 1.57214
Order: 21270, avg: 0.00271273, max: 1.9725


In [246]:
print(order)
print(y_to_user_and_number[order])
prior_orders_products= x[np.where((x[:,0] == y_to_user_and_number[order][0]) & \
           ( x[:,2] >= y_to_user_and_number[order][1] - 5))]
current_user_x = np.zeros((5, 49688))
for p in prior_orders_products:
    current_user_x[p[2] - (y_to_user_and_number[order][1] - 5)][p[3] - 1] = 1


23870
[97906, 19]


In [206]:
x_predict = np.expand_dims(current_user_x, axis=0)
prediction = LSTM_model_predict.predict(x_predict)

In [224]:
prediction

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan]], dtype=float32)

In [212]:
print("Order: " + str(order) + ", avg: " + str(prediction.mean()) + ", max: " + str(prediction.max()))

Order: 10270, avg: 0.00336084, max: 0.166191


In [34]:
for i in range(predictions[0].shape[0]):
    if predictions[1][0][i] > .9:
        print("for item " + str(i+1) + ", order 0 predicted " + str(predictions[0][0][i]) + 
              " and order 1: " + str(predictions[1][0][i]))

IndexError: invalid index to scalar variable.

In [48]:
# Compare predictions to average prediction
for i in range(predictions[1].shape[0]):
    if predictions[1][i] - (avgPrediction[i]/5) > 0:
        print("for item " + str(i+1) + ", order 0 predicted " + str(predictions[1][i]) + 
              ", compared to average " + str(predictions[1][i] - (avgPrediction[i]/5)))

for item 4366, order 0 predicted 0.0188784, compared to average 0.00669155865908
for item 4461, order 0 predicted 0.00772376, compared to average 0.00564251039177
for item 4557, order 0 predicted 0.0161899, compared to average 0.0129519343376
for item 10326, order 0 predicted 0.105415, compared to average 0.0293390870094
for item 10473, order 0 predicted 0.132391, compared to average 0.033247920312
for item 15772, order 0 predicted 0.044383, compared to average 0.0252355471253
for item 17650, order 0 predicted 0.0298767, compared to average 0.00488806143403
for item 18432, order 0 predicted 0.0417545, compared to average 0.0162319235504
for item 25783, order 0 predicted 0.140442, compared to average 0.0471809849143
for item 37687, order 0 predicted 0.124966, compared to average 0.00349747240543
for item 40910, order 0 predicted 0.0150867, compared to average 0.00435758614913


In [50]:
# Print top predictions
for i in range(predictions[1].shape[0]):
    if predictions[1][i] > .3:
        print("for item " + str(i+1) + ", order 0 predicted " + str(predictions[1][i]) + 
              ", compared to average " + str(predictions[1][i] - (avgPrediction[i]/5)))

for item 196, order 0 predicted 0.646737, compared to average -0.532833182812
for item 651, order 0 predicted 0.327271, compared to average -0.166649734974
for item 1463, order 0 predicted 0.495465, compared to average -0.368329566717
for item 4210, order 0 predicted 0.312522, compared to average -0.335149073601
for item 4472, order 0 predicted 0.346879, compared to average -0.354782402515
for item 4605, order 0 predicted 0.455682, compared to average -0.372770303488
for item 4799, order 0 predicted 0.30766, compared to average -0.405022782087
for item 4920, order 0 predicted 0.462867, compared to average -0.312461322546
for item 5077, order 0 predicted 0.599972, compared to average -0.425931215286
for item 5212, order 0 predicted 0.392503, compared to average -0.272786605358
for item 5450, order 0 predicted 0.484065, compared to average -0.337052577734
for item 5785, order 0 predicted 0.381876, compared to average -0.237834089994
for item 5876, order 0 predicted 0.366023, compared to 

In [260]:
predictions[1][0][0]

0.014651672

In [267]:
# 170 and 10270
actual_dict[170]

{5077,
 6236,
 8153,
 11182,
 13176,
 15093,
 17794,
 18394,
 21137,
 25591,
 34582,
 37766,
 39190,
 40354,
 43772,
 43841,
 49593}

In [17]:
actual_dict[10270]

{210, 8277, 29898, 37646, 39180, 39275, 42736, 45066}

In [22]:
predictions[0].flatten()

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [25]:
avgPrediction.shape

(49688,)

In [51]:
# get previous buys by order_id
rebuy_dict[y_to_user_and_number[10270][0]]

{210,
 890,
 1253,
 1359,
 1700,
 1999,
 2180,
 2295,
 2846,
 3142,
 3252,
 4256,
 4605,
 5194,
 5450,
 5746,
 5769,
 6740,
 7021,
 7063,
 7500,
 7978,
 8092,
 8174,
 8193,
 8203,
 8518,
 8854,
 9076,
 9472,
 9666,
 10132,
 10603,
 11005,
 11096,
 11494,
 13147,
 13629,
 14804,
 15290,
 15592,
 15937,
 16185,
 16797,
 16965,
 17108,
 17221,
 17461,
 17794,
 17872,
 17956,
 18370,
 18465,
 18479,
 18531,
 18656,
 19094,
 19677,
 19906,
 20034,
 20319,
 20406,
 20520,
 20549,
 20977,
 21137,
 21376,
 21616,
 21847,
 22142,
 22260,
 22395,
 22504,
 22935,
 23088,
 23236,
 23734,
 24231,
 24720,
 24852,
 25105,
 25407,
 25659,
 25741,
 25878,
 26307,
 26324,
 26668,
 27086,
 27104,
 27156,
 28204,
 28741,
 28842,
 29234,
 29987,
 30583,
 31343,
 31553,
 31824,
 31961,
 31981,
 32665,
 33401,
 33731,
 34358,
 34592,
 34969,
 35217,
 36082,
 36086,
 36772,
 36918,
 37011,
 38067,
 38293,
 38400,
 39180,
 39275,
 39411,
 39430,
 39928,
 40396,
 40706,
 41500,
 42110,
 42421,
 42625,
 42736,
 